<a href="https://colab.research.google.com/github/nevermind78/Data_Analysis/blob/main/EtudeDeCasSQLite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import standard libs
import os

# if on collab pull required subdirectories
if os.getenv("COLAB_RELEASE_TAG"): 
    # install github clone - allows for easy cloning of subdirectories
    !pip install github-clone
    from pathlib import Path 

    # clone datasets
    if not Path('Data/').is_dir():
        !ghclone https://github.com/nevermind78/Data_Analysis/tree/main/Data
    else:
        print('Data/!')

   
# append path for local library, data, and image import
import sys
sys.path.append('./Data')


In [ ]:
import sqlite3
import pandas as pd

# [Enonce](https://github.com/nevermind78/Data_Analysis/blob/main/Data/ETUDEDECAS.pdf)

# Exercice1

### Récuperer les dataFrame à partir des fichiers csv

In [ ]:
df_enseignants = pd.read_csv('Data/enseignants.csv',sep=";")

In [ ]:
df_enseignants 

renommer les colonnes de dataFrame comme dans le diagramme UML 

df_enseignants.columns = ['idEtudiant','prenom','nom','adresse', 'cp','ville','telFixe','telPortable']

In [ ]:
df_enseignants 

In [ ]:
df_etudiants = pd.read_csv('dataBD/etudiants.csv',sep=";")

In [ ]:
df_etudiants

In [ ]:
df_inscriptions = pd.read_csv('dataBD/inscriptions.csv',sep=";")

In [ ]:
df_inscriptions

In [ ]:
df_matieres = pd.read_csv('dataBD/matieres.csv',sep=";") 

In [ ]:
df_matieres

In [ ]:
df_resultats = pd.read_csv('dataBD/resultats.csv',sep=";") 

In [ ]:
df_resultats

### Création de la base et création de la table enseignants

In [ ]:
connection = sqlite3.connect('universite.db.db')
cursor = connection.cursor()
cursor.execute('Create Table if not exists enseignants (idEnseignant INTEGER PRIMARY KEY , nom Text, prenom Text,adresse Text,cp Text, ville Text,telFixe Text , telPortable Text)')
connection.commit()

In [ ]:
df_enseignants.to_sql('enseignants', connection, if_exists='replace', index = False)

index = False pour ne pas ajouter un champ index dans la table (par défaut index=True)

### Création de la table etudiants 

In [ ]:
cursor.execute('Create Table if not exists etudiants (idEtudiant INTEGER PRIMARY KEY , nom Text, prenom Text,adresse Text,cp Text, ville Text,telFixe Text , telPortable Text)')
connection.commit()

In [ ]:
df_etudiants.columns = ['idEtudiant','prenom','nom','adresse', 'cp','ville','telFixe','telPortable']

In [ ]:
df_etudiants.to_sql('etudiants', connection, if_exists='replace', index = False)

### Création de la table inscriptions

In [ ]:
df_inscriptions.columns = ['idInscription','IdEtudiant','codeMatiere']

In [ ]:
df_inscriptions.to_sql('inscriptions', connection, if_exists='replace', index = False)

### Création de la table matieres

In [ ]:
df_matieres.columns = ['codeMatiere','salle','IdEnseignant']

In [ ]:
df_matieres.to_sql('matieres', connection, if_exists='replace', index = False)

### Création de la table resultats

In [ ]:
df_resultats.columns = ['IdResultat','IdInscription','note']

In [ ]:
df_resultats.to_sql('resultats', connection, if_exists='replace', index = False)

### Exercice2

## DataFrame: moyennes par matière ( à partir d'une requête) 

In [ ]:
df_matieres = pd.read_sql_query("SELECT codeMatiere,note FROM resultats r , inscriptions i where r.IdInscription=i.IdInscription", connection)
df_matieres

##### Moyenne de chaque matière

In [ ]:
df_matieres.groupby('codeMatiere')["note"].mean().round(2)

##### Moyenne maximale de chaque matière

In [ ]:
df_matieresMax=df_matieres.groupby('codeMatiere')["note"].max().round(2)

In [ ]:
df_matieresMax

In [ ]:
df_matieresMax.to_json("matiereMax.json")

In [ ]:
df_matieresMax.to_json("matiereMax.json")

##### Moyenne minimale de chaque matière

In [ ]:
df_matieresMin=df_matieres.groupby('codeMatiere')["note"].min().round(2)

In [ ]:
df_matieresMin

##### Moyenne de toute les matières 

In [ ]:
moyenneTotale=df_matieres["note"].mean().round(2)

In [ ]:
moyenneTotale

##### nbEtudiants Inscrit par matière

df_nbEtudiantsParDepartement = pd.read_sql_query("SELECT codeMatiere,note FROM resultats r , inscriptions i where r.IdInscription=i.IdInscription", connection)
df_matieres

In [ ]:
nbEtudiants=pd.read_sql_query("SELECT codeMatiere,idEtudiant FROM inscriptions ", connection)
nbEtudiants

In [ ]:
nbEtudiantInscMat=nbEtudiants.groupby('codeMatiere')["IdEtudiant"].value_counts()

In [ ]:
nbEtudiantInscMat

In [ ]:
nbEtudiantsParMat=pd.read_sql_query("SELECT codeMatiere,COUNT(IdEtudiant)  FROM inscriptions GROUP BY codeMatiere", connection)
nbEtudiantsParMat

In [ ]:
nbEtudiantsParDep=pd.read_sql_query("SELECT cp,COUNT(IdEtudiant)  FROM etudiants GROUP BY cp ", connection)
nbEtudiantsParDep

#### Regrouper les dataframes et écrire dans un fichier JSON